### Challenge
Find the most visited URL per country per day during a week of your choice (e.g. 2018-04-01 until 2018-04-08).

### Expected output
As part of this challenge, we would like you to create a more detailed diagram of the whole system, with every resource reflected on it. Include an explanation on what it is that it does and your approach to solving the challenge.

Feel free to include as well any code or screenshots that are part of your solution if relevant!

### Approach
#### Athena
1. Understand the data using Athena i.e. what the column names mean, what kind of values and data types
2. Choose which columns would be useful - i.e. country_code, url_visited, date
3. Test queries out step by step with the following to try out:
    * COUNT(date) to count how many url per country per day
    * Filter the dates: BETWEEN 2018-04-01 AND 2018-04-08
    * Use GROUP by on country_code, date
    * Do some research to find out how we could get the url that is most visited on the url column
4. Repeat testing until the query performs as expected and gives the output I am looking for

Below is the table I expect to transform to be loaded to my psql table locally

| country_code | date       | most_visited_url                |
|---------|------------|--------------------|
| US    | 2018-04-01 | www.netflix.com |
| CA   | 2018-04-01 | www.netflix.com  |

### The process of optimising my SQL queries in Athena


1. Get the first five rows to understand what kind of values each column contains
```
SELECT *
FROM vod_clickstream
LIMIT 5;
```
* After executing this query, I found that 
    * `server_request_country_code` shows the country code
    * `dt` shows the full date i.e. 2016-06-09
    * `event_url` shows the url visited i.e. https://www.netflix.com/browse

2. Base on the above findings, I decided to narrow down and check out these three columns
```
SELECT 
server_request_country_code, 
dt, 
event_url
FROM 
vod_clickstream
LIMIT 5;
```
Below is the output I got from Athena and I thought it looked good with the info I need find out what's the most visited url for each country per day for a week of my choice
![Image showing the output from Athena after narrowing down to the 3 columns](./athena_outputs/narrow_query.png)

3. After this I decided to try and filter the dates using the BETWEEN clause
```
SELECT 
server_request_country_code, 
dt, 
event_url
FROM 
vod_clickstream
WHERE dt BETWEEN '2018-04-01' AND '2018-04-08'
LIMIT 5;
```
![Image showing the output from Athena after  filtering on the dates to the week of my choice](./athena_outputs/filter_dates_query.png)

4. Next I decided to try group by on the country code and date so that I can see if it includes all the dates I need. I choose to filter on the country code to be Hong Kong so it's easier to see without all the countries. In the process I was reminded that to use GROUP BY you need to have an aggregate function so to be easy, I just decided to use COUNT on the event_url
```
SELECT 
server_request_country_code,  
dt, 
COUNT(event_url) AS total_url_visited
FROM 
vod_clickstream
WHERE dt BETWEEN '2018-04-01' AND '2018-04-08'
AND server_request_country_code = 'HK'
GROUP BY server_request_country_code, dt;
```
![Image showing the output of the GROUP BY query filtering on country being Hong Kong](./athena_outputs/group_by_check_dates_query.png)

This looks great except we need to filter out one of the extra days as there are 8 days above. After further checking, 2018-04-01 is a Sunday and I want to use the week beginning on a Monday so this date will be filtered out now.

5. Adjusted the BETWEEN clause so 2018-04-01 is not included
```
SELECT 
server_request_country_code,  
dt, 
COUNT(event_url) AS total_url_visited
FROM 
vod_clickstream
WHERE dt BETWEEN '2018-04-02' AND '2018-04-08'
AND server_request_country_code = 'HK'
GROUP BY server_request_country_code, dt;
```
![Image showing 2018-04-01 not included in the results anymore](./athena_outputs/optimised_between_clause_query.png)

After doing some research, I found out that I just need to add the url column on to the SELECT and GROUP BY clause so that it uniquely identifies the country code, date and url combinations. 

6. Adjusted the SQL query and aded `event_url` in the SELECT and GROUP BY clause to see if it can identify unique combinations of country code, date and url for Hong Kong. Also decided to rename the ALIAS on the count to `total_visited`
```
SELECT 
server_request_country_code,  
dt, 
event_url,
COUNT(*) AS total_visited
FROM 
vod_clickstream
WHERE dt BETWEEN '2018-04-02' AND '2018-04-08'
AND server_request_country_code = 'HK'
GROUP BY server_request_country_code, dt, event_url;
```
This gave me 6000+ rows and I could definitely see unique combinations of country code, date and url
![Image showing 15 rows of the sql statement used above](./athena_outputs/unique_combinations_query.png) 

7. After some research I think the above query can be used as an inner query and we need an outer query to find the most frequent url. What I think we need to do is SELECT MAX on the `event_url` and `total_visited` columns. Then use a GROUP BY clause on the country code and date.
```
SELECT
    server_request_country_code,
    dt,
    MAX(event_url) AS most_visited_url,
    MAX(total_visited) AS total_visited
FROM (
    SELECT 
        server_request_country_code,  
        dt, 
        event_url,
        COUNT(*) AS total_visited
    FROM 
        vod_clickstream
    WHERE 
        dt BETWEEN '2018-04-02' AND '2018-04-08'
        AND server_request_country_code = 'HK'
    GROUP BY 
        server_request_country_code, dt, event_url
) AS url_counts
GROUP BY 
    server_request_country_code, dt;
```
![Image showing the result after adding MAX in the outer query and using the old query as the inner query](./athena_outputs/using_max_and_subquery.png)

This looks really good, we will just need to work on how to ORDER it in the final query. Let's optimise this query with ORDER by first on the country code and date before getting rid of Hong Kong in the filtering.

8. ADD ORDER BY at the end for country code and date (only date is really needed here but using both to check it works before removing HK)
```
SELECT
    server_request_country_code,
    dt,
    MAX(event_url) AS most_visited_url,
    MAX(total_visited) AS total_visited
FROM (
    SELECT 
        server_request_country_code,  
        dt, 
        event_url,
        COUNT(*) AS total_visited
    FROM 
        vod_clickstream
    WHERE 
        dt BETWEEN '2018-04-02' AND '2018-04-08'
        AND server_request_country_code = 'HK'
    GROUP BY 
        server_request_country_code, dt, event_url
) AS url_counts
GROUP BY 
    server_request_country_code, dt
ORDER BY
    server_request_country_code, dt;
```
![Image showing optimised query output ordering by date](./athena_outputs/order_by_date_query.png)

This looks great, we can it's showing in order by the date. Now let's get rid of HK in the inner query filter. 

9. Removed filter on country code being Hong Kong and I decided to keep the `total_visited` as I think it's useful although not asked by the challenge and not what I had in mind originally from my plan. 
```
SELECT
    server_request_country_code,
    dt,
    MAX(event_url) AS most_visited_url,
    MAX(total_visited) AS total_visited
FROM (
    SELECT 
        server_request_country_code,  
        dt, 
        event_url,
        COUNT(*) AS total_visited
    FROM 
        vod_clickstream
    WHERE 
        dt BETWEEN '2018-04-02' AND '2018-04-08'
    GROUP BY 
        server_request_country_code, dt, event_url
) AS url_counts
GROUP BY 
    server_request_country_code, dt
ORDER BY
    server_request_country_code, dt;
```
![Image showing all countries with the most frequently visited link](./athena_outputs/removed_hk_query.png)

There are questionable data here where I first see some empty values for country code in the first 6 rows and also A1 and A2. I would prefer to get rid of the rows where the country code is NULL first in the inner query in this case. Also have a look at what A1 and A2 country code is as in my domain knowledge, that doesn't look like a country code.

Okay so after some research I found that A1 country code stands for `Anonymous proxy`. What Anonymous proxies are is that it allows users to hide where their traffic is coming from (IP Address) so we don't know which country they are coming from. Let's drop the empty country code and the A1 country code. I couldn't find any information on A2 but as in my domain it doesn't look like a country code, I think it's best to drop this out of the final result.

10. Remove empty string, A1 and A2 through inner query by adding country code `NOT IN ('A1', 'A2', '')`
```
SELECT
    server_request_country_code,
    dt,
    MAX(event_url) AS most_visited_url,
    MAX(total_visited) AS total_visited
FROM (
    SELECT 
        server_request_country_code,  
        dt, 
        event_url,
        COUNT(*) AS total_visited
    FROM 
        vod_clickstream
    WHERE 
        dt BETWEEN '2018-04-02' AND '2018-04-08'
        AND server_request_country_code NOT IN ('A1', 'A2', '')
    GROUP BY 
        server_request_country_code, dt, event_url
) AS url_counts
GROUP BY 
    server_request_country_code, dt
ORDER BY
    server_request_country_code, dt;
```
![Image showing optimised output and getting rid of rows that are not real country codes](./athena_outputs/filter_out_not_real_country_code_query.png)

This looks great and not we are ready to transform and load the data to a psql table locally

In [1]:
import boto3
import time
import psycopg2   
from dotenv import load_dotenv
import os